In [1]:
import pandas as pd
import numpy as np

# Операции над данными в библиотеке Pandas

Для **унитарных операций**, например изменения знака и тригоноетрических функций, при использовании ее универсальных функций в выводе будут **сохранены индекс и метки столбцов**.

А для **бинарных операций**, например сложения и умножения, библиотека Pandas будет автоматически **выравнивать индексы** при передаче объектов универсальной функции.

Это значит, что задачи сохранения контекста данных и объединение данных из различных источников становятся наделжно защищенными от ошибок. 

## Универсальные функции: сохранение индекса

В силу того что библиотека Pandas предназначена для работы с библиотекой NumPy, все универсальные функции библиотеки NumPy будут работать с оъектами Series и DataFrame библиотеки Pandas

In [2]:
rng = np.random.RandomState(42)
series = pd.Series(rng.randint(0, 10, 4))
series

0    6
1    3
2    7
3    4
dtype: int64

In [3]:
df = pd.DataFrame(rng.randint(0, 10, (3, 4)), columns=['A','B','C','D'])
df

,A,B,C,D
0,6,9,2,6
1,7,4,3,7
2,7,2,5,4


Если применить универсальную функцию NumPy к любому из этх объектов, результатом будет другой объект библиотеки Pandas с **сохранением индексов**

In [4]:
np.exp(series)

0     403.428793
1      20.085537
2    1096.633158
3      54.598150
dtype: float64

In [5]:
np.sin(df * np.pi / 4)

,A,B,C,D
0,-1.000000,7.071068e-01,1.000000,-1.000000e+00
1,-0.707107,1.224647e-16,0.707107,-7.071068e-01
2,-0.707107,1.000000e+00,-0.707107,1.224647e-16


## Универсальные функции: выравнивания индексов 

При бинарнх операциях над двумя объектами Series или DataFrame библиотека Pandas будет выравнивать индексы в процессе выполнения операции. Это очень удобно при работе с неполными данными.

### Выравнивание индексов в объектах Series

###### Мини лайфхак: с помощью нижнего подчеркивания можно выделить регистры чисел, чтобы их было проще воспринимать 

Допустим мы объединили два различных источника данных, чтобы найти три штата США с наибольшей **площадью** и три штата США с наибольшим количеством **населения**

In [6]:
area = pd.Series({
    'Alaska': 1_723_337,
    'Texas': 695_662,
    'California': 423_967,
}, name='area')

population = pd.Series({
    'California': 38_332_521,
    'Texas': 26_448_193,
    'New York': 19_651_127,
}, name='population')

Посмотрим, что будет если разделить второй результат на первый для вычисления плотности наслеления:

In [7]:
population / area

Alaska              NaN
California    90.413926
New York            NaN
Texas         38.018740
dtype: float64

Получившийся в итоге масив содержит **объединение индексов** двух исходных массивов

In [8]:
area.index.union(population.index)

Index(['Alaska', 'California', 'New York', 'Texas'], dtype='object')

Все отсутствующие значения по умолчанию заполняются значением **NaN**

In [9]:
A = pd.Series([2, 4, 6], index=[0, 1, 2])
B = pd.Series([1, 3, 5], index=[1, 2, 3])

A + B

0    NaN
1    5.0
2    9.0
3    NaN
dtype: float64

In [10]:
A.add(B, fill_value=0)

0    2.0
1    5.0
2    9.0
3    5.0
dtype: float64

### Выравнивание индексов в объектах DataFrame

При выполнении операций над объектами DataFrame происходит аналогичное выравнивание как для столбцов, так и для индексов

In [11]:
A = pd.DataFrame(rng.randint(0, 20, (2, 2)), 
                 columns=list('AB'), index=list('ab'))
A

,A,B
a,1,11
b,5,1


In [12]:
B = pd.DataFrame(rng.randint(0, 10, (3, 3)), 
                 columns=list('BAC'), 
                 index=list('bcd'))
B

,B,A,C
b,4,0,9
c,5,8,0
d,9,2,6


In [13]:
A + B

,A,B,C
a,NaN,NaN,NaN
b,5.0,5.0,NaN
c,NaN,NaN,NaN
d,NaN,NaN,NaN


Обратите внимание, что индексы выравниваются правильно независимо от их расположения в двух объектах и индексы в полученном результате отсортированы.

Посчитаем среднее значение всех элементов объекта A и используем его для заполнения пустот при сложении датафремов

In [14]:
A.stack() # выстроить значения объекта A в один столбец

a  A     1
   B    11
b  A     5
   B     1
dtype: int64

In [15]:
A.stack().mean()

4.5

In [16]:
fill = A.stack().mean()
A.add(B, fill_value=fill)

,A,B,C
a,5.5,15.5,NaN
b,5.0,5.0,13.5
c,12.5,9.5,4.5
d,6.5,13.5,10.5


Хм, в случае со значением ('a', 'C') пустое значение осталось незаполненым. Думаю, это связано стем, что такой комбинации индекса и столбца нет ни в первом ни во втором датафрейме. Такая логика

#### Соответсвтие между операторами языка Python и методами библиотеки Pandas

| Оператор языка Python | Метод (-ы) библиотеки Pandas |
| :-: | :-: |
| + | add() |
| - | sub(), subtract() |
| * | mul(), multiply() |
| / | truediv(), div(), divide() |
| // | floordiv() |
| % | mod() |
| ** | pow() |

### Универсальные функции: выполнение операций между объектами DataFrame и Series

При выполнении операций между объектами DataFrame и Series выравнивание столбцов происходит аналогичным образом.

Рассмотрим одну из часто встречающихся операций - вычисление разности двумерного массива и одной из его строк:

In [17]:
A = rng.randint(10, size=(3,4))
A

array([[3, 8, 2, 4],
       [2, 6, 4, 8],
       [6, 1, 3, 8]])

In [18]:
A - A[0]

array([[ 0,  0,  0,  0],
       [-1, -2,  2,  4],
       [ 3, -7,  1,  4]])

Вычитание из двумерного массива одной из его строк выполняется построчно.
Аналогично и для объекта DataFrame в библиотеке Pandas

In [19]:
df = pd.DataFrame(A, columns=list('QRST'))
df - df.iloc[0]

,Q,R,S,T
0,0,0,0,0
1,-1,-2,2,4
2,3,-7,1,4


In [20]:
df.subtract(df.loc[0], axis=1)

,Q,R,S,T
0,0,0,0,0
1,-1,-2,2,4
2,3,-7,1,4


Чтобы выполнить эту операцию по столбацам, можно использовать упомянутый метод .substract, указав ключевое слово axis

In [21]:
df.subtract(df['R'], axis=0)

,Q,R,S,T
0,-5,0,-6,-4
1,-4,0,-2,2
2,5,0,2,7


Обратите вимание, что операции DataFrame / Series, аналогично обсуждавшимся ранее операция, будут автоматически выполнять выравнивание индексов мжду двумя их элементами:

In [22]:
df

,Q,R,S,T
0,3,8,2,4
1,2,6,4,8
2,6,1,3,8


In [23]:
halfrow = df.iloc[0, ::2]
halfrow

Q    3
S    2
Name: 0, dtype: int64

In [24]:
df - halfrow

,Q,R,S,T
0,0.0,NaN,0.0,NaN
1,-1.0,NaN,2.0,NaN
2,3.0,NaN,1.0,NaN


Подобное сохранение и выравнивание индексов и столбцов означает, что операции над данными в библиотеке Pandas всегда сохраняют контекст данных, предотвращая возможные ошибки при работе с неоднородными и/или неправильно/неодинаково выровненными данными в исходных массивах NumPy